In [1]:
import pandas as pd
import requests
import bs4 as beautifulsoup
from bs4 import BeautifulSoup
#https://www.nbadraft.net/search-players/ Website we will iterate through

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

base_url = 'http://www.nbadraft.net'


In [2]:
def FetchURL(url):
    r = requests.get(url)
    return r.content

In [3]:
url_list = []
NUM_PAGES_TO_SCRAPE=250

#Creating a list of all links to pages where tables of player links are stored

for i in range(0, NUM_PAGES_TO_SCRAPE):
    url_link = base_url + "/search-players/"
    if i == 0:
        url_list.append(url_link)
    else: 
        url_link = url_link + "?offset="+str(i)
        url_list.append(url_link)
    
#https://www.nbadraft.net/search-players/ ~ first page
#https://www.nbadraft.net/search-players/?offset=210 ~ all other pages

player_links = []

In [4]:
url_list[:5]

['http://www.nbadraft.net/search-players/',
 'http://www.nbadraft.net/search-players/?offset=1',
 'http://www.nbadraft.net/search-players/?offset=2',
 'http://www.nbadraft.net/search-players/?offset=3',
 'http://www.nbadraft.net/search-players/?offset=4']

In [5]:
def GetPlayerURLs(url):
    print(url)
    #Creates the set of player urls of a single page
    soup = BeautifulSoup(FetchURL(url), "html.parser")

    player_list = []

    for a in soup.find_all(href=True):
        #print("Found the URL:", a["href"], a.text)
        if "https://www.nbadraft.net/players/" in a["href"]:
            player_list.append(a["href"])

    player_set = set(player_list) 
    
    return player_set

def AllPlayerURLs(all_urls=url_list):
    """
    Creates the set of all player urls, from all pages
    """
    complete_set = set()
    
    count_page = 0 
    
    for url in all_urls:
            print("Scraping page #:", count_page)
            player_urls = GetPlayerURLs(url)
            if len(player_urls) ==0: #after ~250 pages or so, nbadraft.com runs out of data
                break
            else:
                complete_set.update(player_urls)
                count_page+=1
    
    """
    try:
        for url in all_urls:
            print("Scraping page #:", count_page)
            complete_set.update(GetPlayerURLs(url))
            count_page+=1
    except ErrorName:
        print("Page #", count_page, "doesn't exist.")
    """
    
    return complete_set
    

player_url_list = list(AllPlayerURLs())

Scraping page #: 0
http://www.nbadraft.net/search-players/
Scraping page #: 1
http://www.nbadraft.net/search-players/?offset=1
Scraping page #: 2
http://www.nbadraft.net/search-players/?offset=2
Scraping page #: 3
http://www.nbadraft.net/search-players/?offset=3
Scraping page #: 4
http://www.nbadraft.net/search-players/?offset=4
Scraping page #: 5
http://www.nbadraft.net/search-players/?offset=5
Scraping page #: 6
http://www.nbadraft.net/search-players/?offset=6
Scraping page #: 7
http://www.nbadraft.net/search-players/?offset=7
Scraping page #: 8
http://www.nbadraft.net/search-players/?offset=8
Scraping page #: 9
http://www.nbadraft.net/search-players/?offset=9
Scraping page #: 10
http://www.nbadraft.net/search-players/?offset=10
Scraping page #: 11
http://www.nbadraft.net/search-players/?offset=11
Scraping page #: 12
http://www.nbadraft.net/search-players/?offset=12
Scraping page #: 13
http://www.nbadraft.net/search-players/?offset=13
Scraping page #: 14
http://www.nbadraft.net/searc

Scraping page #: 117
http://www.nbadraft.net/search-players/?offset=117
Scraping page #: 118
http://www.nbadraft.net/search-players/?offset=118
Scraping page #: 119
http://www.nbadraft.net/search-players/?offset=119
Scraping page #: 120
http://www.nbadraft.net/search-players/?offset=120
Scraping page #: 121
http://www.nbadraft.net/search-players/?offset=121
Scraping page #: 122
http://www.nbadraft.net/search-players/?offset=122
Scraping page #: 123
http://www.nbadraft.net/search-players/?offset=123
Scraping page #: 124
http://www.nbadraft.net/search-players/?offset=124
Scraping page #: 125
http://www.nbadraft.net/search-players/?offset=125
Scraping page #: 126
http://www.nbadraft.net/search-players/?offset=126
Scraping page #: 127
http://www.nbadraft.net/search-players/?offset=127
Scraping page #: 128
http://www.nbadraft.net/search-players/?offset=128
Scraping page #: 129
http://www.nbadraft.net/search-players/?offset=129
Scraping page #: 130
http://www.nbadraft.net/search-players/?off

In [6]:
mydf = pd.DataFrame()

In [7]:
s = "123123STRINGabcabc"

def find_between( s, first, last, include_first_last=False):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        if include_first_last==False:
            return s[start:end]
        elif include_first_last==True:
            return first + s[start:end] + last
    except ValueError:
        return ""

def find_between_r( s, first, last ):
    try:
        start = s.rindex( first ) + len( first )
        end = s.rindex( last, start )
        return s[start:end]
    except ValueError:
        return ""


print(find_between(s,"123", "abc"))
print(find_between_r(s,"123","abc"))

123STRING
STRINGabc


In [10]:
def ScrapeProfile(url, df=pd.DataFrame(columns=["Name"])):
    soup = BeautifulSoup(FetchURL(url), "html.parser")
    count=0
    profile_data = []
    for i in soup.find_all('div', {'class': 'wpb_wrapper'}): #looping through most of the html
        #print(i.text) #Debugging
        if count==0: #these data points should be the same for all rows (of the same player)
            player_soup = list(filter(None, i.text.split("\n")))
            #print(player_soup)
            print(len(player_soup))
            name = player_soup[0].split("- ")[1]
            height = player_soup[1]
            weight = player_soup[2]
            position = player_soup[3]
            college = player_soup[4]
            class_status = player_soup[5]
            birthday =  player_soup[7].split(" ")[0]
            hometown=player_soup[9]
            high_school= player_soup[11]
            athleticism=player_soup[17]
            size=player_soup[19]
            defense=player_soup[21]
            strength= player_soup[23]
            quickness=player_soup[25]
            leadership=player_soup[27]
            jump_shot=player_soup[29]
            nba_ready=player_soup[31]
            ball_handling=player_soup[33]
            potential= player_soup[35]
            passing=player_soup[37]
            intangibles=player_soup[39]
            draft_class=player_soup[40][:4]
            try: #Mock and board position aren't always recorded
                mock_position=player_soup[41]
            except:
                mock_position="N/A"
            try:
                big_board_position=player_soup[43]
            except:
                big_board_position="N/A"
            overall= player_soup[-2]
            player_url=url.split("/")[-2]
            
        elif "AnalysisNewsArticlesSocial" in i.text:
            #Scraping the scouting reports from the website...
            player_soup = find_between(i.text, "NBA Comparison: ","News")+"News"
            
            #There may be 1 or more scouting reports per player, so I will need to split them  
            player_soup = player_soup.split("Strengths: ") #splits raw text by who wrote the scouting report
            
            nbadraft_comp = player_soup[0]
    
            for scouting_reports in player_soup[1:]:
                scouting_reports = scouting_reports.split("Weaknesses: ")
                strengths = scouting_reports[0]
                #print("Strengths:", strengths)
                
                scouting_reports = scouting_reports[1].split("Notes: ")
                weaknesses = scouting_reports[0]
                #print("Weaknesses:", weaknesses)
                
                scouting_reports = scouting_reports[1].split("\n")
                notes = scouting_reports[0]
                #print("Notes:", notes)
                
                #print("Scouting Reports:", scouting_reports)
                #scouting_reports[1] is currently a string like "Aran Smith 3/14/15"
                #I want to seperate the names and date, so I split once, from the right in case
                #Player has more than a first and last name I'm not sure if Karl Anthony-Towns would be properly processed
                scouting_reports = scouting_reports[1].rsplit(maxsplit=1)
                author=scouting_reports[0]
                report_date=scouting_reports[1]
                #print("Author:", author)
                #print("Report Date:", report_date)
                

                #Append this scouting report to our dataframe
                df = df.append({"Name": name,
                            "Height": height, 
                            "Weight":weight,
                            "Position": position,
                            "College": college,
                            "ClassStatus": class_status,
                            "Birthday":birthday,
                            "Hometown": hometown,
                            "High School": high_school,
                            "Athleticism": athleticism,
                            "Size": size,
                            "Defense": defense,
                            "Strength": strength,
                            "Quickness": quickness,
                            "Leadership": leadership,
                            "Jump Shot": jump_shot,
                            "NBA Ready": nba_ready,
                            "Ball Handling": ball_handling,
                            "Potential": potential,
                            "Passing": passing,
                            "Intangibles": intangibles,
                            "DraftClass": draft_class,
                            "MockPosition":mock_position,
                            "BigBoardPosition": big_board_position,
                            "Overall": overall,
                            "url": player_url, 
                            "Scouting_Strengths":strengths, 
                            "Scouting_Weaknesses":weaknesses,
                            "Scouting_Notes": notes,
                            "Scouting_Author": author,
                            "Scouting_Report_Date":report_date,
                           },ignore_index=True)
    
            return df
                
            
            
        else:
            pass
           #print(i.text) #Debugging
        
        #print("count", count) #Debugging
        count+=1
                          
    
#ScrapeProfile("https://www.nbadraft.net/players/kris-dunn/")
#ScrapeProfile("https://www.nbadraft.net/players/obi-toppin/")
#ScrapeProfile("https://www.nbadraft.net/players/craig-brackins/") #Not all profiles are scrapable. Fix this in future versions
#ScrapeProfile("https://www.nbadraft.net/players/aaron-gray/")

ScrapeProfile("https://www.nbadraft.net/players/alan-williams/")

44


,Name,Athleticism,Ball Handling,BigBoardPosition,Birthday,ClassStatus,College,Defense,DraftClass,Height,High School,Hometown,Intangibles,Jump Shot,Leadership,MockPosition,NBA Ready,Overall,Passing,Position,Potential,Quickness,Scouting_Author,Scouting_Notes,Scouting_Report_Date,Scouting_Strengths,Scouting_Weaknesses,Size,Strength,Weight,url
0,Alan Williams,6,8,Overall:,01/28/93,Senior,UC Santa Barbara,7,Big,"6-8,",North,"Phoenix, AZ",7,7,8,81,8,86,8,Power Forward/Center,6,6,Jacob Stallard,"Measured 6’8 (in shoes) 264 lbs, with a 7’1.5 ...",6/13/15,Dominant big man who put up some outrageous nu...,The knocks against Williams are obvious. Firs...,6,9,260,alan-williams


In [12]:
mydf = pd.DataFrame(columns=["Name"])
count=0
for pages in player_url_list:
    print("Scraping profile...", count, "out of", len(player_url_list), "total prospects." )
    try:
        profile = ScrapeProfile(pages)
        #print(profile)
        mydf = mydf.append(profile, ignore_index=True)
    except IndexError: 
        print(pages, "is unscrapable.")
        
    count+=1
    

Scraping profile... 0 out of 6374 total prospects.
41
Scraping profile... 1 out of 6374 total prospects.
38
https://www.nbadraft.net/players/dwayne-mitchell/ is unscrapable.
Scraping profile... 2 out of 6374 total prospects.
36
https://www.nbadraft.net/players/jamal-crawford/ is unscrapable.
Scraping profile... 3 out of 6374 total prospects.
41
https://www.nbadraft.net/players/jamal-bey/ is unscrapable.
Scraping profile... 4 out of 6374 total prospects.
34
https://www.nbadraft.net/players/oktay-ulubay/ is unscrapable.
Scraping profile... 5 out of 6374 total prospects.
42
Scraping profile... 6 out of 6374 total prospects.
46
https://www.nbadraft.net/players/chandler-parsons/ is unscrapable.
Scraping profile... 7 out of 6374 total prospects.
39
https://www.nbadraft.net/players/shane-edwards/ is unscrapable.
Scraping profile... 8 out of 6374 total prospects.
35
https://www.nbadraft.net/players/robbie-beran/ is unscrapable.
Scraping profile... 9 out of 6374 total prospects.
38
https://www.

41
Scraping profile... 100 out of 6374 total prospects.
46
Scraping profile... 101 out of 6374 total prospects.
44
https://www.nbadraft.net/players/tyler-harvey/ is unscrapable.
Scraping profile... 102 out of 6374 total prospects.
41
Scraping profile... 103 out of 6374 total prospects.
41
Scraping profile... 104 out of 6374 total prospects.
41
Scraping profile... 105 out of 6374 total prospects.
37
https://www.nbadraft.net/players/nick-blake/ is unscrapable.
Scraping profile... 106 out of 6374 total prospects.
42
Scraping profile... 107 out of 6374 total prospects.
42
https://www.nbadraft.net/players/fletcher-magee/ is unscrapable.
Scraping profile... 108 out of 6374 total prospects.
44
https://www.nbadraft.net/players/isaac-humphries/ is unscrapable.
Scraping profile... 109 out of 6374 total prospects.
41
Scraping profile... 110 out of 6374 total prospects.
37
https://www.nbadraft.net/players/dieonte-miles/ is unscrapable.
Scraping profile... 111 out of 6374 total prospects.
40
https:

46
https://www.nbadraft.net/players/kevin-murphy/ is unscrapable.
Scraping profile... 197 out of 6374 total prospects.
37
https://www.nbadraft.net/players/okben-ulubay/ is unscrapable.
Scraping profile... 198 out of 6374 total prospects.
44
Scraping profile... 199 out of 6374 total prospects.
41
Scraping profile... 200 out of 6374 total prospects.
41
Scraping profile... 201 out of 6374 total prospects.
35
https://www.nbadraft.net/players/josh-smith/ is unscrapable.
Scraping profile... 202 out of 6374 total prospects.
41
Scraping profile... 203 out of 6374 total prospects.
36
https://www.nbadraft.net/players/dusan-tanaskovic/ is unscrapable.
Scraping profile... 204 out of 6374 total prospects.
46
Scraping profile... 205 out of 6374 total prospects.
38
https://www.nbadraft.net/players/omer-asik/ is unscrapable.
Scraping profile... 206 out of 6374 total prospects.
41
Scraping profile... 207 out of 6374 total prospects.
44
https://www.nbadraft.net/players/patric-young/ is unscrapable.
Scra

46
Scraping profile... 287 out of 6374 total prospects.
40
https://www.nbadraft.net/players/lucas-dias-silva/ is unscrapable.
Scraping profile... 288 out of 6374 total prospects.
35
https://www.nbadraft.net/players/muon-reath/ is unscrapable.
Scraping profile... 289 out of 6374 total prospects.
41
Scraping profile... 290 out of 6374 total prospects.
41
Scraping profile... 291 out of 6374 total prospects.
36
https://www.nbadraft.net/players/franck-seguela/ is unscrapable.
Scraping profile... 292 out of 6374 total prospects.
41
Scraping profile... 293 out of 6374 total prospects.
35
https://www.nbadraft.net/players/reese-dixon/ is unscrapable.
Scraping profile... 294 out of 6374 total prospects.
46
Scraping profile... 295 out of 6374 total prospects.
41
Scraping profile... 296 out of 6374 total prospects.
46
https://www.nbadraft.net/players/jarrett-allen/ is unscrapable.
Scraping profile... 297 out of 6374 total prospects.
42
https://www.nbadraft.net/players/jalen-lecque/ is unscrapable.

42
https://www.nbadraft.net/players/vernon-carey/ is unscrapable.
Scraping profile... 380 out of 6374 total prospects.
39
https://www.nbadraft.net/players/lazar-zivanovic/ is unscrapable.
Scraping profile... 381 out of 6374 total prospects.
41
Scraping profile... 382 out of 6374 total prospects.
41
Scraping profile... 383 out of 6374 total prospects.
37
https://www.nbadraft.net/players/kartal-ozmizrak/ is unscrapable.
Scraping profile... 384 out of 6374 total prospects.
41
Scraping profile... 385 out of 6374 total prospects.
46
Scraping profile... 386 out of 6374 total prospects.
29
https://www.nbadraft.net/players/jimma-gatwech/ is unscrapable.
Scraping profile... 387 out of 6374 total prospects.
41
https://www.nbadraft.net/players/curtis-jones/ is unscrapable.
Scraping profile... 388 out of 6374 total prospects.
36
https://www.nbadraft.net/players/jordan-burger/ is unscrapable.
Scraping profile... 389 out of 6374 total prospects.
46
Scraping profile... 390 out of 6374 total prospects

...

39
https://www.nbadraft.net/players/anthony-lamb/ is unscrapable.
Scraping profile... 6007 out of 6374 total prospects.
39
https://www.nbadraft.net/players/pablo-perez/ is unscrapable.
Scraping profile... 6008 out of 6374 total prospects.
35
https://www.nbadraft.net/players/abdoulaye-mbaye/ is unscrapable.
Scraping profile... 6009 out of 6374 total prospects.
41
Scraping profile... 6010 out of 6374 total prospects.
41
Scraping profile... 6011 out of 6374 total prospects.
39
https://www.nbadraft.net/players/bruno-vrcic/ is unscrapable.
Scraping profile... 6012 out of 6374 total prospects.
40
https://www.nbadraft.net/players/evan-fournier/ is unscrapable.
Scraping profile... 6013 out of 6374 total prospects.
38
https://www.nbadraft.net/players/james-augustine/ is unscrapable.
Scraping profile... 6014 out of 6374 total prospects.
41
Scraping profile... 6015 out of 6374 total prospects.
46
https://www.nbadraft.net/players/travis-leslie/ is unscrapable.
Scraping profile... 6016 out of 6374 

46
Scraping profile... 6098 out of 6374 total prospects.
35
https://www.nbadraft.net/players/jordan-wilmore/ is unscrapable.
Scraping profile... 6099 out of 6374 total prospects.
43
Scraping profile... 6100 out of 6374 total prospects.
39
https://www.nbadraft.net/players/jordan-fouse/ is unscrapable.
Scraping profile... 6101 out of 6374 total prospects.
40
https://www.nbadraft.net/players/boban-marjanovic/ is unscrapable.
Scraping profile... 6102 out of 6374 total prospects.
39
https://www.nbadraft.net/players/dmytro-skapintsev/ is unscrapable.
Scraping profile... 6103 out of 6374 total prospects.
38
https://www.nbadraft.net/players/jason-richardson/ is unscrapable.
Scraping profile... 6104 out of 6374 total prospects.
39
https://www.nbadraft.net/players/brian-beard/ is unscrapable.
Scraping profile... 6105 out of 6374 total prospects.
37
https://www.nbadraft.net/players/abdou-ndiaye/ is unscrapable.
Scraping profile... 6106 out of 6374 total prospects.
40
https://www.nbadraft.net/play

41
Scraping profile... 6188 out of 6374 total prospects.
41
Scraping profile... 6189 out of 6374 total prospects.
37
https://www.nbadraft.net/players/jameel-brown/ is unscrapable.
Scraping profile... 6190 out of 6374 total prospects.
35
https://www.nbadraft.net/players/jwan-roberts/ is unscrapable.
Scraping profile... 6191 out of 6374 total prospects.
41
Scraping profile... 6192 out of 6374 total prospects.
46
Scraping profile... 6193 out of 6374 total prospects.
44
Scraping profile... 6194 out of 6374 total prospects.
35
https://www.nbadraft.net/players/jay-heath/ is unscrapable.
Scraping profile... 6195 out of 6374 total prospects.
39
https://www.nbadraft.net/players/lamond-murray-jr/ is unscrapable.
Scraping profile... 6196 out of 6374 total prospects.
35
https://www.nbadraft.net/players/deandre-wilkins/ is unscrapable.
Scraping profile... 6197 out of 6374 total prospects.
41
Scraping profile... 6198 out of 6374 total prospects.
37
https://www.nbadraft.net/players/jabri-abdur-rahim/

41
Scraping profile... 6281 out of 6374 total prospects.
41
Scraping profile... 6282 out of 6374 total prospects.
35
https://www.nbadraft.net/players/justice-ajogbor/ is unscrapable.
Scraping profile... 6283 out of 6374 total prospects.
39
https://www.nbadraft.net/players/kostyantyn-anikiyenko/ is unscrapable.
Scraping profile... 6284 out of 6374 total prospects.
39
https://www.nbadraft.net/players/malik-fitts/ is unscrapable.
Scraping profile... 6285 out of 6374 total prospects.
37
https://www.nbadraft.net/players/damion-baugh/ is unscrapable.
Scraping profile... 6286 out of 6374 total prospects.
46
https://www.nbadraft.net/players/kentavious-caldwell-pope/ is unscrapable.
Scraping profile... 6287 out of 6374 total prospects.
44
https://www.nbadraft.net/players/cameron-moore/ is unscrapable.
Scraping profile... 6288 out of 6374 total prospects.
41
Scraping profile... 6289 out of 6374 total prospects.
39
https://www.nbadraft.net/players/carlos-hidalgo/ is unscrapable.
Scraping profile.

46
https://www.nbadraft.net/players/montrezl-harrell/ is unscrapable.
Scraping profile... 6367 out of 6374 total prospects.
43
Scraping profile... 6368 out of 6374 total prospects.
46
Scraping profile... 6369 out of 6374 total prospects.
46
https://www.nbadraft.net/players/cameron-bairstow/ is unscrapable.
Scraping profile... 6370 out of 6374 total prospects.
41
Scraping profile... 6371 out of 6374 total prospects.
44
https://www.nbadraft.net/players/bj-young/ is unscrapable.
Scraping profile... 6372 out of 6374 total prospects.
41
Scraping profile... 6373 out of 6374 total prospects.
41


In [13]:
url_list

['http://www.nbadraft.net/search-players/',
 'http://www.nbadraft.net/search-players/?offset=1',
 'http://www.nbadraft.net/search-players/?offset=2',
 'http://www.nbadraft.net/search-players/?offset=3',
 'http://www.nbadraft.net/search-players/?offset=4',
 'http://www.nbadraft.net/search-players/?offset=5',
 'http://www.nbadraft.net/search-players/?offset=6',
 'http://www.nbadraft.net/search-players/?offset=7',
 'http://www.nbadraft.net/search-players/?offset=8',
 'http://www.nbadraft.net/search-players/?offset=9',
 'http://www.nbadraft.net/search-players/?offset=10',
 'http://www.nbadraft.net/search-players/?offset=11',
 'http://www.nbadraft.net/search-players/?offset=12',
 'http://www.nbadraft.net/search-players/?offset=13',
 'http://www.nbadraft.net/search-players/?offset=14',
 'http://www.nbadraft.net/search-players/?offset=15',
 'http://www.nbadraft.net/search-players/?offset=16',
 'http://www.nbadraft.net/search-players/?offset=17',
 'http://www.nbadraft.net/search-players/?offse

In [14]:
mydf

,Athleticism,Ball Handling,BigBoardPosition,Birthday,ClassStatus,College,Defense,DraftClass,Height,High School,Hometown,Intangibles,Jump Shot,Leadership,MockPosition,NBA Ready,Name,Overall,Passing,Position,Potential,Quickness,Scouting_Author,Scouting_Notes,Scouting_Report_Date,Scouting_Strengths,Scouting_Weaknesses,Size,Strength,Weight,url
0,7,7,70,07/01/91,Senior,Bucknell,7,2013,"6-11,",Roseville,"Roseville, MN",8,8,7,50,8,Mike Muscala,88,8,Power Forward/Center,7,7,Aran Smith,Viewed by scouts as a likely draft pick and pr...,3/28/13,"Bigman with solid fluidity, mobility and footw...","While a fluid athlete, he lacks the explosiven...",8,6,230,mike-muscala
1,9,7,8,03/14/93,Freshman,UNLV,7,2013,"6-7,",Findlay Prep,"Brampton, Ontario, Canada",8,8,8,10,8,Anthony Bennett,94,7,Small Forward/Power Forward,9,7,@LamontPeterson,"Measured 6’7 (in shoes) 242 lbs, with a 7’1 wi...",2/1/13,Bennett has expanded his game offensively and ...,Somewhat of a tweener as he is slightly unders...,8,8,240,anthony-bennett
2,9,7,6,03/26/96,Freshman,Duke,8,2015,"6-6,",St. John’s,"Houston, TX",8,8,7,7,8,Justise Winslow,97,7,Shooting Guard/Small Forward,9,9,Aran Smith,"Measured 6’6, 229.1 lbs, with a 6’10 wingspan ...",1/8/15,Big time athlete with elite level speed and at...,Throughout his high school and AAU career he f...,8,9,222,justise-winslow
3,7,6,Overall:,03/28/91,Senior,Tennessee,7,Big,"6-5,",Liberty County,"Midway, GA",7,8,7,82,8,Jordan McRae,85,7,Shooting Guard,7,7,Jorrye Nixon,"Measured 6’5 (in shoes) 175 lbs, with a 7’0 wi...",5/26/14,McRae is a tall and long wing prospect who pla...,Though McRae has good height and exceptional l...,8,6,180,jordan-mcrae
4,8,7,8,08/30/96,Junior,Villanova,9,2018,"6-7,",Great Valley,"Malvern, PA",9,9,7,10,9,Mikal Bridges,95,6,Shooting Guard/Small Forward,7,8,Aran Smith,"Born and raised in Philadelphia, PA… Has 3 bro...",3/11/18,Prototypical 3-and-D wing for the next level …...,The biggest knock on Bridges is his lack of st...,9,7,210,mikal-bridges
5,6,7,Overall:,02/12/92,Senior,Ohio St.,8,Big,"6-2,",Liberty Benton,"Findlay, OH",7,5,8,97,7,Aaron Craft,84,8,Point Guard,6,7,"Overall: Craft seems to lack the size, athleti...",Named National Defensive Player of the year by...,…,Excellent on-ball defender with an incredible ...,Doesn’t have NBA measurable or athleticism. H...,7,8,192,aaron-craft
6,7,9,9,05/12/98,Freshman,Texas,9,2018,"7-0,",Westtown School,"New York, NY",9,7,8,5,7,Mohamed Bamba,96,7,Center,9,7,Evan Tomes,Committed to University of Texas on 5/18/2017 ...,5/18/17,7’0.5 big man with incredible length (7’9.5 wi...,Minimal offensive skills … Gets a high majorit...,10,7,225,mohamed-bamba
7,8,7,Overall:,06/20/98,Sophomore,Syracuse,8,Big,"6-8,",Athlete Institute,"Mississauga, Ontario, Canada",8,6,7,52,7,Oshae Brissett,89,7,Small Forward,8,7,Ben Yokoyama,"Measured: 6′ 7” barefoot, 6′ 8” in shoes, 203....",6/19/19,A physical player that can rebound the ball … ...,Somewhat limited offensively … Poor 3-point sh...,8,8,205,oshae-brissett
8,8,8,Overall:,05/09/95,International,France,6,Big,"6-7,",Team Site Profile,"Cannes, France",15,7,8,32,7,Timothe Luwawu,90,7,Shooting Guard/Small Forward,7,7,Stefanos Makris,He started his professional career at the age ...,6/21/16,Versatile wing … Great size and wingspan (meas...,Has to bulk up to be able to play at the next ...,8,8,195,timothe-luwawu
9,8,8,Overall:,11/16/94,Senior,Arkansas,8,Big,"6-10,",Huntington Prep,"Abuja, Nigeria",8,7,7,76,7,Moses Kingsley,89,6,Center,7,7,Brandon Jefferson,"Born in Enugu, Nigeria and moved to Mississipp...",3/9/16,Kingsley is a player that looks the part of an...,Still somewhat new to the game (6 years of com...,8,8,230,moses-kingsley


In [15]:
mydf.to_csv("nbadrafttextdata.csv")